In [41]:
# pictures similarity  comparison

In [1]:
import torch
from torchvision import models
from torch import nn
from torchsummary import summary
import numpy as np

In [43]:
# check GPU

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
"cuda" if torch.cuda.is_available() else "cpu"

'cpu'

In [3]:
# load VGG 16 模型
model = models.vgg16(pretrained=True)
model._modules

c:\Users\a4022\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\a4022\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


OrderedDict([('features',
              Sequential(
                (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                (1): ReLU(inplace=True)
                (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                (3): ReLU(inplace=True)
                (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
                (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                (6): ReLU(inplace=True)
                (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                (8): ReLU(inplace=True)
                (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
                (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                (11): ReLU(inplace=True)
                (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                (13): ReL

In [4]:
model.modules

<bound method Module.modules of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2,

In [5]:
print(type(model.modules))
print(type(model._modules))

<class 'method'>
<class 'collections.OrderedDict'>


In [6]:
# remove nn layer after "avgpool" modules

class new_model(nn.Module):
    def __init__(self, pretrained, output_layer):
        super().__init__()
        self.output_layer = output_layer
        self.pretrained = pretrained
        self.children_list = []
        
        # get each modules
        for n,c in self.pretrained.named_children():
            self.children_list.append(c)
            # find specific modules and stop
            if n == self.output_layer:
                print('found !!')
                break

        # build new model
        self.net = nn.Sequential(*self.children_list)
        self.pretrained = None
        
    def forward(self,x):
        x = self.net(x)
        return x
    
model = new_model(model, 'avgpool')
model = model.to(device)   
model._modules

found !!


OrderedDict([('pretrained', None),
             ('net', Sequential(
                (0): Sequential(
                  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                  (1): ReLU(inplace=True)
                  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                  (3): ReLU(inplace=True)
                  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
                  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                  (6): ReLU(inplace=True)
                  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                  (8): ReLU(inplace=True)
                  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
                  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                  (11): ReLU(inplace=True)
                  (12): Conv2d(256, 256, k

In [7]:
model.modules

<bound method Module.modules of new_model(
  (pretrained): None
  (net): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), paddin

In [8]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_sta

In [9]:
model.children_list

[Sequential(
   (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU(inplace=True)
   (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (3): ReLU(inplace=True)
   (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (6): ReLU(inplace=True)
   (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (8): ReLU(inplace=True)
   (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (11): ReLU(inplace=True)
   (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (13): ReLU(inplace=True)
   (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (15): ReLU(inplace=True)
   (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (17): Conv2d(

In [10]:
len(model.children_list)

2

In [11]:
# 任選一張圖片，例如老虎側面照，取得圖檔的特徵向量
from PIL import Image
from torchvision import transforms

filename = './images_test/tiger2.jpg'
input_image = Image.open(filename)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])
input_tensor = transform(input_image)
input_batch = input_tensor.unsqueeze(0).to(device) # 增加一維(筆數)

# 預測
model.eval()
with torch.no_grad():
    output = model(input_batch)
output    

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.2631, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [2.1441, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.3814],
          [1.5381, 0.0000, 0.0000,  ..., 0.0000, 2.0274, 5.3855],
          [0.0000, 0.0000, 0.0000,  ..., 0.5574, 2.1842, 6.1950]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.3603, 0.6468],
          [0.6322, 0.0000, 0.0000,  ..., 0.0000, 1.9587, 1.9612],
          [2.0919, 0.3300, 0.0000,  ..., 0.0000, 0.0000, 1.4273],
          ...,
          [0.0000, 2.7205, 5.2899,  ..., 2.0377, 2.2244, 1.3909],
          [0.0000, 3.1269, 3.3711,  ..., 0.4258, 1.1930, 0.6181],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 2.0133,  ..., 0

In [12]:
print(output.shape)

torch.Size([1, 512, 7, 7])


In [13]:
# using cosine_similarity to compare eigenvector

In [14]:
## step1 : get all .jpg files in images_test folder
from os import listdir
from os.path import isfile, join

# 取得 images_test 目錄下所有 .jpg 檔案名稱
img_path = './images_test/'
image_files = np.array([f for f in listdir(img_path) 
        if isfile(join(img_path, f)) and f[-3:] == 'jpg'])
image_files

array(['astronaut.jpg', 'bird.jpg', 'bird2.jpg', 'cat.jpg', 'daisy1.jpg',
       'daisy2.jpg', 'deer.jpg', 'elephant.jpg', 'elephant2.jpg',
       'lion1.jpg', 'lion2.jpg', 'owl.jpg', 'panda1.jpg', 'panda2.jpg',
       'panda3.jpg', 'peacock.jpg', 'rose2.jpg', 'tiger1.jpg',
       'tiger2.jpg', 'tiger3.jpg', 'toucan.jpg'], dtype='<U13')

In [15]:
## step2 : get all .jpg pixels in images_test folder
import os

# 合併所有圖檔
model.eval()
X = torch.tensor([])
for filename in image_files:
    input_image = Image.open(os.path.join(img_path, filename))
    input_tensor = transform(input_image)
    input_batch = input_tensor.unsqueeze(0).to(device) # 增加一維(筆數)
    if len(X.shape) == 1:
        # print(input_batch.shape)
        X = input_batch
    else:
        # print(input_batch.shape)
        X = torch.cat((X, input_batch), dim=0)

In [16]:
X.shape

torch.Size([21, 3, 224, 224])

In [17]:
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [19]:
from torchinfo import summary
summary(model, input_size=(3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20)

c:\Users\a4022\anaconda3\lib\site-packages\torchinfo\torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
c:\Users\a4022\anaconda3\lib\site-packages\torch\storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                   Input Shape          Output Shape         Param #              Trainable
new_model                                [3, 224, 224]        [512, 7, 7]          --                   True
├─Sequential: 1-1                        [3, 224, 224]        [512, 7, 7]          --                   True
│    └─Sequential: 2-1                   [3, 224, 224]        [512, 7, 7]          --                   True
│    │    └─Conv2d: 3-1                  [3, 224, 224]        [64, 224, 224]       1,792                True
│    │    └─ReLU: 3-2                    [64, 224, 224]       [64, 224, 224]       --                   --
│    │    └─Conv2d: 3-3                  [64, 224, 224]       [64, 224, 224]       36,928               True
│    │    └─ReLU: 3-4                    [64, 224, 224]       [64, 224, 224]       --                   --
│    │    └─MaxPool2d: 3-5               [64, 224, 224]       [64, 112, 112]       --                   --
│    │    └─Conv2d: 

In [20]:
# predict all pictures
with torch.no_grad():
    features = model(X)
features.shape

torch.Size([21, 512, 7, 7])

In [25]:
## step4 : use cosine_similarity function
from sklearn.metrics.pairwise import cosine_similarity

# comparison : Tiger2.jpg with other pictures
no=-2
print(image_files[no])

# 轉為二維向量，類似扁平層(Flatten)
features2 = features.cpu().reshape((features.shape[0], -1))
print("features2 shape: ", features2.size())

# 排除 Tiger2.jpg 的其他圖檔特徵向量
other_features = np.concatenate((features2[:no], features2[no+1:]))
print("other_features shape: ", other_features.shape)

# 使用 cosine_similarity 計算 Cosine 函數
similar_list = cosine_similarity(features2[no:no+1], other_features, 
                                 dense_output=False)

# 顯示相似度，由大排到小
print(np.sort(similar_list[0])[::-1])

# 依相似度，由大排到小，顯示檔名
image_files2 = np.delete(image_files, no)
image_files2[np.argsort(similar_list[0])[::-1]]

tiger3.jpg
features2 shape:  torch.Size([21, 25088])
other_features shape:  (20, 25088)
[0.28471893 0.27424297 0.20017293 0.13029028 0.124294   0.12173368
 0.0922957  0.09078282 0.09055958 0.08343987 0.08006315 0.07478783
 0.07359645 0.07357086 0.07160097 0.06039139 0.04973827 0.04559418
 0.04373307 0.03809744]


array(['tiger2.jpg', 'tiger1.jpg', 'lion2.jpg', 'owl.jpg', 'peacock.jpg',
       'panda1.jpg', 'deer.jpg', 'cat.jpg', 'lion1.jpg', 'panda2.jpg',
       'elephant2.jpg', 'astronaut.jpg', 'elephant.jpg', 'panda3.jpg',
       'bird.jpg', 'daisy2.jpg', 'rose2.jpg', 'toucan.jpg', 'daisy1.jpg',
       'bird2.jpg'], dtype='<U13')

In [27]:
similar_list.shape

(1, 20)

In [28]:
# comparison：bird.jpg with other pictures
no=1
print(image_files[no])


# 使用 cosine_similarity 計算 Cosine 函數
other_features = np.concatenate((features2[:no], features2[no+1:]))
similar_list = cosine_similarity(features2[no:no+1], other_features, 
                                 dense_output=False)

# 顯示相似度，由大排到小
print(np.sort(similar_list[0])[::-1])

# 依相似度，由大排到小，顯示檔名
image_files2 = np.delete(image_files, no)
image_files2[np.argsort(similar_list[0])[::-1]]

bird.jpg
[0.25217512 0.2471562  0.21036279 0.17145817 0.1494664  0.14713255
 0.14371438 0.10901471 0.09888166 0.09075938 0.08877531 0.08840889
 0.08423284 0.07160099 0.06278276 0.06060669 0.05814058 0.05581865
 0.05143056 0.04135138]


array(['toucan.jpg', 'owl.jpg', 'panda3.jpg', 'bird2.jpg', 'deer.jpg',
       'cat.jpg', 'panda1.jpg', 'tiger2.jpg', 'lion2.jpg', 'tiger1.jpg',
       'peacock.jpg', 'rose2.jpg', 'elephant2.jpg', 'tiger3.jpg',
       'panda2.jpg', 'lion1.jpg', 'astronaut.jpg', 'daisy2.jpg',
       'elephant.jpg', 'daisy1.jpg'], dtype='<U13')